## Machine Learning 실습

### 타이타닉 탑승객 생존 예측 Classification with Python

In [2]:
!pip install tensorflow
# 1. 데이터 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#classification(분류)
from sklearn.tree import DecisionTreeClassifier      #의사결정트리
from sklearn.ensemble import RandomForestClassifier  #랜덤포레스트
from sklearn.neighbors import KNeighborsClassifier   #KNN(K_Nearst_Neighbor)K-최근접이웃
from sklearn.linear_model import LogisticRegression  #로지스틱회귀
from sklearn.svm import SVC


from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score


  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=bec303e5fe43e30a599201a6c41bc6d5b5e2f63aa271668a3beef72eccaa7267
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=9784f89f2af2e478a1467497d41254dfb62fbda4a22c00283a3d7cf9e73d5294
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built termcolor wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


distutils: c:\users\admin\anaconda3\Include\UNKNOWN
sysconfig: c:\users\admin\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\admin\anaconda3\Include\UNKNOWN
sysconfig: c:\users\admin\anaconda3\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\admin\anaconda3\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 2. 데이터 확인하기
df = pd.read_csv('../ML/03Titanic_dataset.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

In [ ]:
# 3. 빠진 값 확인
df.isnull().sum()

In [ ]:
# 4. 사용하지 않을 feature 제거
feature = df.drop(['survived','name', 'ticket','boat','body','home.dest','cabin'], axis=1)
feature

In [ ]:
# SimpleImputer로 만든 함수 가져오기
from sklearn.impute import SimpleImputer

# numeric(fare, age) -mean
mean_imputer = SimpleImputer(strategy = 'mean')

def f_mean_imputer(df, i):
    mean_imputer.fit(feature.iloc[:, i:i+1]) # 해당 열의 평균으로 대체
    feature.iloc[:, i:i+1] = mean_imputer.transform(df.iloc[:, i:i+1])
    return df.isnull().sum()

# Clean Missing Data 2 - string(embarked) -most_frequent
mode_imputer = SimpleImputer(strategy = 'most_frequent')
def f_mode_imputer(df, i):
    mode_imputer.fit(feature.iloc[:,:i+1]) # 해당 열의 최빈값으로 대체
    feature.iloc[:, :i+1] = mode_imputer.transform(df.iloc[:,:i+1])
    return df.isnull().sum()

In [ ]:
# 5. Impute - Fare
f_mean_imputer(feature, 5)

In [ ]:
# 6. Impute - Age
f_mean_imputer(feature, 2)

In [ ]:
# 7. Impute - Embarked
f_mode_imputer(feature, 6)

In [ ]:
# 8. Feature Heatmap 시각화에 앞서 모듈 가져오기
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 8. Feature Heatmap 시각화
# feature heatmap 그려보기
featurecorr = feature.corr()
plt.figure(figsize=(15,10))
sns.heatmap(featurecorr, cmap='viridis')
#sns.heatmap(df_17corr, annot=True, fmt='f')
plt.title('Heatmap of featurecorr by seaborn', fontsize=15)

plt.show()

In [ ]:
# 8-1. 시각화 - 성별에 따른 생존자 수

plt.figure(figsize=(10,5))
sns.countplot('survived', hue='gender', data=df)

In [ ]:
# 8-2 시각화 - 선실 등급에 따른 생존여부

plt.figure(figsize=(10,5))
sns.countplot('survived', hue='embarked', data=df)

In [ ]:
# 8-3 시각화 - 자매여부에 따른 생존여부

plt.figure(figsize=(10,5))
sns.countplot('survived', hue='sibsp', data=df)

In [ ]:
# 8-4 시각화 - 부모자식에 따른 생존여부

plt.figure(figsize=(10,5))
sns.countplot('survived', hue='parch', data=df)

In [ ]:
# 9. X/y  분리
# 종속 변수를 담는 듯
y = df['survived']

# 필요없는 변수들을 제거하고 x축 독립변수들로 만듬
#x = transform_features(feature)

In [ ]:
# 10. 데이터 변환(one hot encoding)
# gender 문자열이니까 숫자로 바꿔주기
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(),[1,6])], remainder='passthrough')

feature = ct.fit_transform(feature)

print(feature)

In [ ]:
# 11. 훈련셋/평가셋 분리


# train_test_split으로 데이터를 나눠줌
# test_size로 train과 test 데이터의 비율을 설정할 수 있음 (default = 0.25)
# train_size도 설정가능 (default = 1-test_size)
# random_state로 셔플을 위한 seed 값을 지정할 수 있음(int로도 입력가능)
# shuffle도 설정가능. 데이터 섞을건지 (default = True)
# stratify도 설정가능 하며 classification을 다룰때 필요함. 한쪽에 치중되는 것을 방지 (default = None)
x_train, x_test, y_train, y_test = train_test_split(feature, y,
                                                    test_size=0.3,
                                                    random_state=372)

In [ ]:
# 12. 모델 학습
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
log_clf = LogisticRegression()
XGB_clf = XGBClassifier()

rf_clf.fit(x_train, y_train)

y_pred = rf_clf.predict(x_test)

In [ ]:
# 13. 모델 성능 확인(evaluate)



mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)


print('mae = ',mae)
print('mse = ',mse)
print('r2 = ',r2)
print('accuracy = ',acc)
print('precision = ',pre)
print('recall = ',rec)

In [ ]:
# 13-1. confusion matrix 확인
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(rf_clf, x_train, y_train, cv=3)
cf = confusion_matrix(y_train, y_train_pred)
print(cf)

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

😊